## DataFrames: operacións avanzadas

Agora que xa se coñecen as operacións básicas con DataFrames, é o momento de ver operacións máis complexas e habituais en proxectos reais, como por exemplo:

- joins
- agrupacións complexas
- operacións con xanelas (window functions)
- rollups
- cubos
- pivotes

Nalgúns exemplos (especialmente cando hai datas en formato texto), pode ser necesario axustar a configuración da sesión de Spark para que o parseo e o tratamento de datas sexa consistente.


In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("04-Dataframes-02") \
.config("spark.sql.session.timeZone", "UTC") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()
sc = spark.sparkContext

# Amosamos a configuración aplicada
print("Zona horaria da sesión (spark.sql.session.timeZone):")
print(spark.conf.get("spark.sql.session.timeZone"))

print("Política de parseo de datas (spark.sql.legacy.timeParserPolicy):")
print(spark.conf.get("spark.sql.legacy.timeParserPolicy"))

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hadoop/.ivy2/cache
The jars for the packages stored in: /home/hadoop/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4e105918-a4af-4746-995c-fcf6d4b1a7f4;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.7 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.fi

Zona horaria da sesión (spark.sql.session.timeZone):
UTC
Política de parseo de datas (spark.sql.legacy.timeParserPolicy):
LEGACY


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 36462)
Traceback (most recent call last):
  File "/usr/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/usr/local/lib/python3.8/dist-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/lib/python3.8/dist-packages/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
  File "/usr/local/lib/python3.8/dist-packages/pyspark/accumulators.py", line 271, in accum_updates
    num_updates = read_int(self.rf

## Joins en DataFrames

Os **joins** permiten combinar datos de dous DataFrames a partir dunha condición, normalmente baseada nunha ou varias columnas comúns. O funcionamento é equivalente aos joins das bases de datos relacionais, pero aplicado a DataFrames distribuídos.

Antes de ver os distintos tipos de join, créanse uns DataFrames de exemplo cos que se vai traballar.


In [2]:
# Exemplo: Crear DataFrames de exemplo para traballar con joins

# DataFrame de persoas
# - id: identificador da persoa
# - name: nome
# - graduate: identificador do programa (clave de unión)
# - marks: lista de identificadores de status
person = (
    spark.createDataFrame([
        (0, "Bill Chambers", 0, [100]),
        (1, "Matel Zaharla", 1, [500, 250, 100]),
        (2, "Michael Armbrust", 1, [250, 100])
    ])
    .toDF("id", "name", "graduate", "marks")
)

print("DataFrame person:")
person.show()

# DataFrame cos programas de posgrao
# - id: identificador do programa (clave de unión)
# - degree: tipo de título
# - department: departamento
# - school: universidade
graduate_program = (
    spark.createDataFrame([
        (0, "Masters", "School of Information", "UC Berkeley"),
        (2, "Masters", "EECS", "UC Berkeley"),
        (1, "Ph. D.", "EECS", "UC Berkeley")
    ])
    .toDF("id", "degree", "department", "school")
)

print("DataFrame graduate_program:")
graduate_program.show()


# DataFrame cos status asociados ás marcas
spark_status = (
    spark.createDataFrame([
        (500, "Vice President"),
        (250, "PMC Member"),
        (100, "Contributor")
    ])
    .toDF("id", "status")
)

print("DataFrame spark_status:")
spark_status.show()




DataFrame person:


+---+----------------+--------+---------------+
| id|            name|graduate|          marks|
+---+----------------+--------+---------------+
|  0|   Bill Chambers|       0|          [100]|
|  1|   Matel Zaharla|       1|[500, 250, 100]|
|  2|Michael Armbrust|       1|     [250, 100]|
+---+----------------+--------+---------------+

DataFrame graduate_program:


+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1| Ph. D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+

DataFrame spark_status:


[Stage 5:>                                                          (0 + 1) / 1]

+---+--------------+
| id|        status|
+---+--------------+
|500|Vice President|
|250|    PMC Member|
|100|   Contributor|
+---+--------------+



### Inner join

O **inner join** avalía a condición de unión en ambos DataFrames e devolve **unicamente** as filas nas que a avaliación é verdadeira, é dicir, aquelas filas que teñen correspondencia nos dous DataFrames.

As filas que non teñen clave coincidente en algún dos DataFrames **non aparecen** no resultado final.

É o tipo de join máis habitual cando se quere traballar só cos rexistros que existen en ambos conxuntos de datos.


In [26]:
# Exemplo: Inner join entre person e graduate_program usando unha expresión de unión

# Definimos a condición de unión:
# - person.graduate contén o id do programa asociado á persoa
# - graduate_program.id é o id do programa
join_expression = person["graduate"] == graduate_program["id"]

# Realizamos o inner join:
# Só se devolverán as filas nas que exista coincidencia entre person.graduate e graduate_program.id
resultado = person.join(graduate_program, join_expression)

# Imprimimos a descrición do resultado
print("Inner join entre person e graduate_program (só filas con coincidencia na clave):")

# Amosamos o resultado do join
resultado.show()


Inner join entre person e graduate_program (só filas con coincidencia na clave):
+---+----------------+--------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate|          marks| id| degree|          department|     school|
+---+----------------+--------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|       0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matel Zaharla|       1|[500, 250, 100]|  1| Ph. D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|       1|     [250, 100]|  1| Ph. D.|                EECS|UC Berkeley|
+---+----------------+--------+---------------+---+-------+--------------------+-----------+



### Outer join (full outer)

O **outer join** (ou *full outer join*) avalía a condición de unión en ambos DataFrames e devolve:

- as filas nas que a condición é verdadeira (hai coincidencia),
- e tamén as filas nas que a condición é falsa (non hai coincidencia).

Cando unha fila dun dos DataFrames non ten correspondencia no outro, os campos correspondentes complétanse con `NULL`.

Este tipo de join é útil cando se quere **conservar toda a información de ambos DataFrames**, mesmo cando non hai coincidencia entre eles.


In [ ]:
# Exemplo: Full outer join entre person e graduate_program

# Indicamos explicitamente o tipo de join
join_type = "outer"

# Realizamos o outer join:
# Mantéñense todas as filas de ambos DataFrames, usando NULL onde non exista valor
resultado = person.join(graduate_program, join_expression, join_type)

# Imprimimos a descrición do resultado
print("Outer join entre person e graduate_program (mantén todas as filas):")

# Amosamos o resultado do join
resultado.show()


### Left outer join

O **left outer join** avalía a condición de unión en ambos DataFrames e devolve:

- **todas as filas** do DataFrame da esquerda (`person`)
- unidas ás filas do DataFrame da dereita (`graduate_program`) **só cando hai coincidencia** (condición verdadeira)

Cando unha fila do DataFrame esquerdo non ten correspondencia no dereito, os campos do DataFrame dereito complétanse con `NULL`.

Este tipo de join é útil cando se quere manter como referencia o DataFrame da esquerda e engadir información do dereito cando exista.


In [ ]:
# Exemplo: Left outer join entre person e graduate_program

# Indicamos explicitamente o tipo de join
join_type = "left_outer"

# Realizamos o left outer join:
# Mantéñense todas as filas de person, e só se engaden datos de graduate_program cando hai match
resultado = person.join(graduate_program, join_expression, join_type)

# Imprimimos a descrición do resultado
print("Left outer join entre person e graduate_program (mantén todas as filas do esquerdo):")

# Amosamos o resultado do join
resultado.show()


### Right outer join

O **right outer join** avalía a condición de unión en ambos DataFrames e devolve:

- **todas as filas** do DataFrame da dereita (`graduate_program`)
- unidas ás filas do DataFrame da esquerda (`person`) **só cando hai coincidencia** (condición verdadeira)

Cando unha fila do DataFrame dereito non ten correspondencia no esquerdo, os campos do DataFrame esquerdo complétanse con `NULL`.

Este tipo de join é útil cando se quere manter como referencia o DataFrame da dereita e engadir información do esquerdo cando exista.


In [ ]:
# Exemplo: Right outer join entre person e graduate_program

# Indicamos explicitamente o tipo de join
join_type = "right_outer"

# Realizamos o right outer join:
# Mantéñense todas as filas de graduate_program, e só se engaden datos de person cando hai match
resultado = person.join(graduate_program, join_expression, join_type)

# Imprimimos a descrición do resultado
print("Right outer join entre person e graduate_program (mantén todas as filas do dereito):")

# Amosamos o resultado do join
resultado.show()


### Left semi join

O **left semi join** é un tipo especial de join que **non incorpora columnas do segundo DataFrame**.  
Limítase a devolver as filas do **primeiro DataFrame** (`person`) que **teñen correspondencia** no segundo (`graduate_program`) segundo a condición de unión.

Características importantes:
- só se mostran columnas do DataFrame esquerdo
- serve para comprobar existencia de coincidencia (equivalente a un `WHERE EXISTS` en SQL)
- é máis eficiente que un join normal cando só interesa saber se hai match


In [ ]:
# Exemplo: Left semi join entre person e graduate_program

# Indicamos explicitamente o tipo de join
join_type = "left_semi"

# Realizamos o left semi join:
# Devolve só as filas de person que teñen match en graduate_program
resultado = person.join(graduate_program, join_expression, join_type)

# Imprimimos a descrición do resultado
print("Left semi join entre person e graduate_program (só filas do DataFrame esquerdo con match):")

# Amosamos o resultado do join
resultado.show()


### Left anti join

O **left anti join** é o oposto do *left semi join*.  
Devolve as filas do **primeiro DataFrame** (`person`) que **non teñen correspondencia** no segundo (`graduate_program`) segundo a condición de unión.

Características importantes:
- só se mostran columnas do DataFrame esquerdo
- serve para obter “os que non existen no outro DataFrame” (equivalente a un `WHERE NOT EXISTS` en SQL)
- é útil para detectar rexistros orfos, discrepancias entre fontes ou faltas de referencia


In [ ]:
# Exemplo: Left anti join entre person e graduate_program

# Indicamos explicitamente o tipo de join
join_type = "left_anti"

# Realizamos o left anti join:
# Devolve só as filas de person que NON teñen match en graduate_program
resultado = person.join(graduate_program, join_expression, join_type)

# Imprimimos a descrición do resultado
print("Left anti join entre person e graduate_program (filas do esquerdo sen match):")

# Amosamos o resultado do join
resultado.show()


### Cross join

O **cross join** realiza un **produto cartesiano** entre dous DataFrames.  
Cada fila do primeiro DataFrame combínase con **todas** as filas do segundo DataFrame.

O número de filas do resultado será:
`n_filas_df1 × n_filas_df2`

Este tipo de join non utiliza ningunha condición de unión.


In [ ]:
# Exemplo: Cross join entre person e graduate_program

# Realizamos un produto cartesiano entre ambos DataFrames
resultado = person.crossJoin(graduate_program)

# Imprimimos a descrición do resultado
print("Cross join entre person e graduate_program (produto cartesiano):")

# Amosamos o resultado do join
resultado.show()


### Nota de rendemento (broadcast join)

Cando un dos DataFrames é pequeno (por exemplo, táboas de referencia), é recomendable empregar un **broadcast join** para evitar shuffle.

Spark copiará o DataFrame pequeno a todos os executores.


In [27]:
# Exemplo: Broadcast join cando graduate_program é pequeno

from pyspark.sql.functions import broadcast

resultado = person.join(
    broadcast(graduate_program),
    join_expression,
    "inner"
)

print("Inner join usando broadcast para optimizar rendemento:")
resultado.show()


Inner join usando broadcast para optimizar rendemento:
+---+----------------+--------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate|          marks| id| degree|          department|     school|
+---+----------------+--------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|       0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matel Zaharla|       1|[500, 250, 100]|  1| Ph. D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|       1|     [250, 100]|  1| Ph. D.|                EECS|UC Berkeley|
+---+----------------+--------+---------------+---+-------+--------------------+-----------+



## Operacións con xanelas (window functions)

As **operacións con xanelas** permiten calcular valores agregados mantendo as filas orixinais. Isto é especialmente útil cando se quere obter métricas “por grupo” (por exemplo por país, por cliente, por día…) sen perder o detalle de cada rexistro.

A diferenza dun `groupBy()` clásico:
- nun `groupBy()`, as filas agrúpanse e o resultado devolve **unha fila por grupo** (redúcese o número de filas).
- nunha función de xanela, calcúlase un valor para **cada fila de entrada**, tomando como referencia un conxunto de filas relacionado con ela, ao que se chama **marco** (frame).

Cada fila pode pertencer a máis dun marco, por exemplo en cálculos por intervalo temporal (medias móbiles), onde a fila dun día pode entrar nos marcos de varios días adxacentes.

![Funcións con xanelas](./images/windows.png)

A especificación dunha xanela defínese normalmente con dúas partes:
- `partitionBy(...)`: define como se agrupan as filas (equivalente á clave do groupBy, pero sen colapsar filas)
- `orderBy(...)`: define a orde dentro de cada partición (habitual cando se traballa con tempo ou ranking)

A continuación prepárase un DataFrame engadindo unha columna `date` a partir de `InvoiceDate`, eliminando horas e minutos.


In [3]:
# Exemplo: Cargar o DataFrame base e preparalo para traballar con xanelas engadindo unha columna date

from pyspark.sql.functions import col, to_timestamp, date_format

# Cargamos os CSV e inferimos o esquema
df_ag = (
    spark.read.format("csv")
        .option("header", "true")
        .option("inferSchema", "true")
        .load("hdfs:///data/retail-data/all/*.csv")
        .coalesce(5)
)

# Gardamos en caché para reutilización en exemplos posteriores
df_ag.cache()

# Rexistramos unha vista temporal (opcional) para consultas SQL
df_ag.createOrReplaceTempView("dfTable")

# Amosamos o esquema para ver os tipos das columnas
print("Esquema de df_ag:")
df_ag.printSchema()


Esquema de df_ag:
root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [4]:
# Exemplo: Crear unha columna 'date' sen hora/minuto a partir de InvoiceDate

from pyspark.sql.functions import col, to_timestamp, date_format

# Convertimos InvoiceDate a timestamp usando un patrón explícito e despois formateámolo como 'yyyy-MM-dd'
df_con_fecha = df_ag.withColumn(
    "date",
    date_format(
        to_timestamp(col("InvoiceDate"), "MM/d/yyyy H:mm"),
        "yyyy-MM-dd"
    )
)

# Eliminamos a columna InvoiceDate para quedarnos só coa data simplificada
df_con_fecha = df_con_fecha.drop("InvoiceDate")

# Rexistramos unha vista temporal para traballar tamén con SQL se se quere
df_con_fecha.createOrReplaceTempView("dfConFecha")

# Amosamos unha mostra para comprobar o resultado
print("Mostra de df_con_fecha (con columna date e sen InvoiceDate):")
df_con_fecha.show(5)


Mostra de df_con_fecha (con columna date e sen InvoiceDate):


[Stage 9:>                                                          (0 + 1) / 1]

+---------+---------+--------------------+--------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|UnitPrice|CustomerID|       Country|      date|
+---------+---------+--------------------+--------+---------+----------+--------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|     2.55|     17850|United Kingdom|2010-12-01|
|   536365|    71053| WHITE METAL LANTERN|       6|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84406B|CREAM CUPID HEART...|       8|     2.75|     17850|United Kingdom|2010-12-01|
|   536365|   84029G|KNITTED UNION FLA...|       6|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|     3.39|     17850|United Kingdom|2010-12-01|
+---------+---------+--------------------+--------+---------+----------+--------------+----------+
only showing top 5 rows



O primeiro paso para empregar unha función de xanela é crear unha **especificación de xanela** (*window specification*).

- `partitionBy(...)` non está relacionado co concepto de particións (repartition/coalesce) visto antes. Aquí úsase para indicar como se “dividen” as filas en **grupos lóxicos** sobre os que se aplicará a xanela.
- `orderBy(...)` determina a orde das filas dentro de cada partición lóxica.
- `rowsBetween(...)` define que filas se inclúen no **marco** (*frame*) para cada fila de entrada. É dicir, indica o rango de filas (anteriores, seguintes, etc.) que se teñen en conta ao calcular unha agregación para esa fila.

No seguinte exemplo o marco inclúe todas as filas desde o inicio da partición (*unboundedPreceding*) ata a fila actual (*currentRow*). Isto permite calcular agregacións “acumuladas” segundo a orde definida.


In [5]:
# Exemplo: Definir unha especificación de xanela para CustomerId e date ordenando por Quantity (desc)

from pyspark.sql.window import Window
from pyspark.sql.functions import col, desc

# Definimos a xanela:
# - Partición lóxica por CustomerId e date (cada cliente e día)
# - Orde interna por Quantity descendente (máis cantidade primeiro)
# - Marco: desde o inicio da partición ata a fila actual
windowSpec = (
    Window
        .partitionBy("CustomerId", "date")
        .orderBy(desc("Quantity"))
        .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

print("WindowSpec definido: partitionBy(CustomerId, date), orderBy(Quantity desc), rowsBetween(unboundedPreceding, currentRow)")


WindowSpec definido: partitionBy(CustomerId, date), orderBy(Quantity desc), rowsBetween(unboundedPreceding, currentRow)


O seguinte paso é definir unha **agregación de xanela**. Neste caso, calcúlase a cantidade máxima (`max`) dentro do marco definido pola xanela.

Ao empregar `.over(windowSpec)`, a función de agregación pasa a calcularse **para cada fila**, en vez de devolver unha única fila por grupo como faría un `groupBy()`.


In [6]:
# Exemplo: Definir unha agregación de xanela (máxima cantidade) usando over(windowSpec)

from pyspark.sql.functions import max

# Creamos unha expresión de columna que calcula o máximo de Quantity dentro do marco da xanela
maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)

print("Columna de xanela definida: maxPurchaseQuantity = max(Quantity) over(windowSpec)")


Columna de xanela definida: maxPurchaseQuantity = max(Quantity) over(windowSpec)


As funcións `rank()` e `dense_rank()` permiten crear un ranking dentro de cada partición lóxica seguindo a orde definida no `orderBy(...)`.

- `rank()`: se hai empates, asigna o mesmo rango e deixa “ocos” na secuencia.
- `dense_rank()`: se hai empates, asigna o mesmo rango pero non deixa “ocos”.

Ambas devolven columnas (expresións) que se poden empregar en `select()`.


In [7]:
# Exemplo: Definir rank e dense_rank sobre a xanela

from pyspark.sql.functions import dense_rank, rank

# Definimos as columnas de ranking segundo a xanela
purchaseRank = rank().over(windowSpec)
purchaseDenseRank = dense_rank().over(windowSpec)

print("Columnas de xanela definidas: purchaseRank e purchaseDenseRank")


Columnas de xanela definidas: purchaseRank e purchaseDenseRank


Antes de aplicar as xanelas, pode amosarse unha mostra do DataFrame para comprobar que existen as columnas `CustomerId`, `date` e `Quantity`. A continuación aplícanse as columnas de xanela nun `select()` para visualizar os valores calculados.

Neste exemplo:
- fíltranse filas con `CustomerId` non nulo
- ordénase por `CustomerId` para facilitar a lectura
- selecciónanse columnas orixinais e columnas calculadas por xanela


In [8]:
# Exemplo: Amosar unha mostra do DataFrame de entrada

print("Mostra de df_con_fecha:")
df_con_fecha.show(5)


Mostra de df_con_fecha:
+---------+---------+--------------------+--------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|UnitPrice|CustomerID|       Country|      date|
+---------+---------+--------------------+--------+---------+----------+--------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|     2.55|     17850|United Kingdom|2010-12-01|
|   536365|    71053| WHITE METAL LANTERN|       6|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84406B|CREAM CUPID HEART...|       8|     2.75|     17850|United Kingdom|2010-12-01|
|   536365|   84029G|KNITTED UNION FLA...|       6|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|     3.39|     17850|United Kingdom|2010-12-01|
+---------+---------+--------------------+--------+---------+----------+--------------+----------+
only showing top 5 rows



In [9]:
# Exemplo: Aplicar columnas de xanela nun select() para ver rank, dense_rank e máximo por marco

resultado = (
    df_con_fecha.where(col("CustomerId").isNotNull())
        .orderBy("CustomerId")
        .select(
            col("CustomerId"),
            col("date"),
            col("Quantity"),
            purchaseRank.alias("quantityRank"),
            purchaseDenseRank.alias("quantityDenseRank"),
            maxPurchaseQuantity.alias("maxPurchaseQuantity")
        )
)

print("Valores de xanela por CustomerId e date (rank, dense_rank, maxPurchaseQuantity):")
resultado.show()


Valores de xanela por CustomerId e date (rank, dense_rank, maxPurchaseQuantity):


[Stage 14:>                                                         (0 + 1) / 1]

+----------+----------+--------+------------+-----------------+-------------------+
|CustomerId|      date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----------+--------+------------+-----------------+-------------------+
|     12346|2011-01-18|   74215|           1|                1|              74215|
|     12346|2011-01-18|  -74215|           2|                2|              74215|
|     12347|2010-12-07|      36|           1|                1|                 36|
|     12347|2010-12-07|      30|           2|                2|                 36|
|     12347|2010-12-07|      24|           3|                3|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|             

Un dos usos máis habituais das funcións de xanela é calcular **valores acumulados**, por exemplo:
- vendas acumuladas
- cantidades acumuladas por cliente
- métricas ao longo do tempo

Isto non se pode facer cun `groupBy()` clásico sen perder o detalle das filas.

In [22]:
# Exemplo: Calcular a cantidade acumulada por Country ao longo do tempo

from pyspark.sql.window import Window
from pyspark.sql.functions import sum, col

# Definimos unha xanela por país, ordenada por data
window_acumulado = (
    Window
        .partitionBy("Country")
        .orderBy("date")
        .rowsBetween(Window.unboundedPreceding, Window.currentRow)
)

# Calculamos a suma acumulada de Quantity
df_acumulado = (
    df_con_fecha.withColumn(
        "quantity_acumulada",
        sum(col("Quantity")).over(window_acumulado)
    )
)

print("Cantidade acumulada por país ao longo do tempo:")
df_acumulado.select("Country", "date", "Quantity", "quantity_acumulada").show(10)


Cantidade acumulada por país ao longo do tempo:


[Stage 62:>                                                         (0 + 1) / 1]

+---------+----------+--------+------------------+
|  Country|      date|Quantity|quantity_acumulada|
+---------+----------+--------+------------------+
|Australia|2010-12-01|       6|                 6|
|Australia|2010-12-01|       8|                14|
|Australia|2010-12-01|      12|                26|
|Australia|2010-12-01|       6|                32|
|Australia|2010-12-01|       4|                36|
|Australia|2010-12-01|       6|                42|
|Australia|2010-12-01|       3|                45|
|Australia|2010-12-01|       2|                47|
|Australia|2010-12-01|       4|                51|
|Australia|2010-12-01|       4|                55|
+---------+----------+--------+------------------+
only showing top 10 rows



Outro caso moi habitual é querer **un único rexistro por grupo**, por exemplo:
- a última compra dun cliente
- o valor máximo por día
- o rexistro máis recente por clave

Para isto emprégase `row_number()` xunto cunha xanela.

In [23]:
# Exemplo: Obter a última compra por CustomerId (a máis recente por data)

from pyspark.sql.functions import row_number

# Definimos unha xanela por cliente, ordenada por data descendente
window_ultimo = (
    Window
        .partitionBy("CustomerId")
        .orderBy(col("date").desc())
)

# Calculamos o número de fila dentro de cada partición
df_rank = df_con_fecha.withColumn(
    "row_num",
    row_number().over(window_ultimo)
)

# Quedámonos só coa fila máis recente de cada cliente
ultimas_compras = df_rank.where(col("row_num") == 1)

print("Última compra por cliente:")
ultimas_compras.select("CustomerId", "date", "Quantity").show(10)


Última compra por cliente:


[Stage 63:=============================>                            (1 + 1) / 2]

+----------+----------+--------+
|CustomerId|      date|Quantity|
+----------+----------+--------+
|      NULL|2011-12-09|       2|
|     12346|2011-01-18|   74215|
|     12347|2011-12-07|      12|
|     12348|2011-09-25|     120|
|     12349|2011-11-21|       2|
|     12350|2011-02-02|      12|
|     12352|2011-11-03|      12|
|     12353|2011-05-19|       2|
|     12354|2011-04-21|      10|
|     12355|2011-05-09|      24|
+----------+----------+--------+
only showing top 10 rows



## Grouping sets

Ata agora víronse agrupacións “simples” nas que se agrega usando un único conxunto fixo de columnas (por exemplo `GROUP BY customerId, stockCode`). Porén, ás veces é necesario obter **varios niveis de agregación nunha única consulta**, por exemplo:

- agregación por (customerId, stockCode)
- e tamén un total xeral sen ningunha columna de agrupación

Para iso existen os **grouping sets**, unha ferramenta de baixo nivel de SQL que permite combinar distintos conxuntos de agrupación dentro dun mesmo `GROUP BY`. O resultado inclúe filas correspondentes a cada conxunto indicado.

A idea é:
- `GROUPING SETS ((a,b), (a), ())`
  - agrega por (a,b)
  - agrega por (a)
  - agrega total xeral con `()`

Nos exemplos seguintes prepárase primeiro un DataFrame sen nulos nas columnas clave e créase unha vista temporal para consultar con SQL.


In [10]:
# Exemplo: Preparar un DataFrame sen nulos nas columnas clave e crear unha vista temporal

from pyspark.sql.functions import col

# Eliminamos filas con nulos nas columnas que se van usar no GROUP BY
# (CustomerId pode ter nulos no dataset, polo que se filtra explicitamente)
df_no_nulo = df_con_fecha.where(
    col("CustomerId").isNotNull() & col("StockCode").isNotNull()
)

# Rexistramos a vista temporal para consultas SQL
df_no_nulo.createOrReplaceTempView("dfNoNulo")

print("Mostra de dfNoNulo (sen nulos en CustomerId e StockCode):")
df_no_nulo.show(5)


Mostra de dfNoNulo (sen nulos en CustomerId e StockCode):
+---------+---------+--------------------+--------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|UnitPrice|CustomerID|       Country|      date|
+---------+---------+--------------------+--------+---------+----------+--------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|     2.55|     17850|United Kingdom|2010-12-01|
|   536365|    71053| WHITE METAL LANTERN|       6|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84406B|CREAM CUPID HEART...|       8|     2.75|     17850|United Kingdom|2010-12-01|
|   536365|   84029G|KNITTED UNION FLA...|       6|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|     3.39|     17850|United Kingdom|2010-12-01|
+---------+---------+--------------------+--------+---------+----------+--------------+----------+
only showing top 5 rows



### Agrupación estándar (sen grouping sets)

Nesta consulta agrúpase por `CustomerId` e `StockCode` e calcúlase a suma de `Quantity`. O resultado contén unha fila por combinación (CustomerId, StockCode).


In [11]:
# Exemplo: GROUP BY estándar por CustomerId e StockCode

print("Suma de Quantity por (CustomerId, StockCode) usando GROUP BY estándar:")
spark.sql("""
    SELECT
        CustomerId,
        StockCode,
        sum(Quantity) AS total_quantity
    FROM dfNoNulo
    GROUP BY CustomerId, StockCode
    ORDER BY CustomerId DESC, StockCode DESC
""").show()


Suma de Quantity por (CustomerId, StockCode) usando GROUP BY estándar:


[Stage 18:=============================>                            (1 + 1) / 2]

+----------+---------+--------------+
|CustomerId|StockCode|total_quantity|
+----------+---------+--------------+
|     18287|    85173|            48|
|     18287|   85040A|            48|
|     18287|   85039B|           120|
|     18287|   85039A|            96|
|     18287|    84920|             4|
|     18287|    84584|             6|
|     18287|   84507C|             6|
|     18287|   72351B|            24|
|     18287|   72351A|            24|
|     18287|   72349B|            60|
|     18287|    47422|            24|
|     18287|    47421|            48|
|     18287|    35967|            36|
|     18287|    23445|            20|
|     18287|    23378|            24|
|     18287|    23376|            48|
|     18287|    23310|            36|
|     18287|    23274|            12|
|     18287|    23272|            12|
|     18287|    23269|            36|
+----------+---------+--------------+
only showing top 20 rows



### O mesmo empregando `GROUPING SETS`

Se se usa `GROUPING SETS((CustomerId, StockCode))` o resultado é equivalente ao `GROUP BY` estándar anterior. Isto serve para entender que `GROUPING SETS` permite especificar explicitamente cal é o conxunto (ou conxuntos) de agrupación que se queren combinar.


In [12]:
# Exemplo: GROUPING SETS cun único conxunto (equivalente ao GROUP BY normal)

print("O mesmo resultado usando GROUPING SETS((CustomerId, StockCode)):")
spark.sql("""
    SELECT
        CustomerId,
        StockCode,
        sum(Quantity) AS total_quantity
    FROM dfNoNulo
    GROUP BY CustomerId, StockCode
    GROUPING SETS ((CustomerId, StockCode))
    ORDER BY total_quantity DESC, CustomerId DESC, StockCode DESC
""").show()


O mesmo resultado usando GROUPING SETS((CustomerId, StockCode)):


[Stage 19:=============================>                            (1 + 1) / 2]

+----------+---------+--------------+
|CustomerId|StockCode|total_quantity|
+----------+---------+--------------+
|     13256|    84826|         12540|
|     17949|    22197|         11692|
|     16333|    84077|         10080|
|     16422|    17003|         10077|
|     16333|    21915|          8120|
|     16308|    16014|          8000|
|     17306|    22616|          6624|
|     18102|    22189|          5946|
|     12901|    84077|          5712|
|     14609|    18007|          5586|
|     12931|    22197|          5340|
|     17450|    22469|          5286|
|     12931|    84879|          5048|
|     14646|    23084|          4801|
|     16029|    22693|          4800|
|     16210|    21137|          4728|
|     17381|    22616|          4704|
|     15769|   85099B|          4700|
|     12901|    21787|          4632|
|     14646|    22629|          4492|
+----------+---------+--------------+
only showing top 20 rows



### Engadir o total xeral con `GROUPING SETS((CustomerId, StockCode), ())`

Se se quere obter tamén o **total xeral** (sen agrupar por ningunha columna), engádese o conxunto baleiro `()` dentro de `GROUPING SETS`.

Isto fai que no resultado apareza unha fila adicional co total de `Quantity` de todo o DataFrame. Nesa fila, as columnas `CustomerId` e `StockCode` aparecen como `NULL`, xa que non forman parte do conxunto de agrupación dese nivel.

Este tipo de resultado (agregación por combinación + total xeral na mesma consulta) é precisamente unha das utilidades principais dos grouping sets.


In [13]:
# Exemplo: GROUPING SETS para obter por (CustomerId, StockCode) e tamén o total xeral

print("Suma por (CustomerId, StockCode) e total xeral usando GROUPING SETS((...), ()):")
spark.sql("""
    SELECT
        CustomerId,
        StockCode,
        sum(Quantity) AS total_quantity
    FROM dfNoNulo
    GROUP BY CustomerId, StockCode
    GROUPING SETS ((CustomerId, StockCode), ())
    ORDER BY total_quantity DESC, CustomerId DESC, StockCode DESC
""").show()


Suma por (CustomerId, StockCode) e total xeral usando GROUPING SETS((...), ()):


[Stage 22:=============================>                            (1 + 1) / 2]

+----------+---------+--------------+
|CustomerId|StockCode|total_quantity|
+----------+---------+--------------+
|      NULL|     NULL|       4906888|
|     13256|    84826|         12540|
|     17949|    22197|         11692|
|     16333|    84077|         10080|
|     16422|    17003|         10077|
|     16333|    21915|          8120|
|     16308|    16014|          8000|
|     17306|    22616|          6624|
|     18102|    22189|          5946|
|     12901|    84077|          5712|
|     14609|    18007|          5586|
|     12931|    22197|          5340|
|     17450|    22469|          5286|
|     12931|    84879|          5048|
|     14646|    23084|          4801|
|     16029|    22693|          4800|
|     16210|    21137|          4728|
|     17381|    22616|          4704|
|     15769|   85099B|          4700|
|     12901|    21787|          4632|
+----------+---------+--------------+
only showing top 20 rows



Cando se usan `GROUPING SETS`, `ROLLUP` ou `CUBE`, aparecen valores `NULL` que representan subtotais.  
Para poder **identificar explicitamente** que tipo de fila é cada unha, Spark ofrece a función `grouping()`.

- `grouping(col) = 0` → a columna participa na agrupación
- `grouping(col) = 1` → a columna é un subtotal

In [24]:
# Exemplo: Identificar subtotais nun GROUPING SETS usando grouping()

from pyspark.sql.functions import grouping

resultado = spark.sql("""
    SELECT
        CustomerId,
        StockCode,
        sum(Quantity) AS total_quantity,
        grouping(CustomerId) AS g_customer,
        grouping(StockCode) AS g_stock
    FROM dfNoNulo
    GROUP BY CustomerId, StockCode
    GROUPING SETS ((CustomerId, StockCode), ())
""")

print("Grouping sets con indicadores de subtotal:")
resultado.show()


Grouping sets con indicadores de subtotal:


[Stage 66:=============================>                            (1 + 1) / 2]

+----------+---------+--------------+----------+-------+
|CustomerId|StockCode|total_quantity|g_customer|g_stock|
+----------+---------+--------------+----------+-------+
|     15358|    22935|            12|         0|      0|
|     14646|    22624|            81|         0|      0|
|     13873|    23243|             4|         0|      0|
|     13873|    21391|            12|         0|      0|
|     14534|    22470|             3|         0|      0|
|     18237|    23298|            15|         0|      0|
|     12431|    23268|            12|         0|      0|
|     17049|    22355|            30|         0|      0|
|     16115|    22993|            27|         0|      0|
|     13799|    22522|            12|         0|      0|
|     12656|    22899|            50|         0|      0|
|     14498|   84030E|             1|         0|      0|
|     15023|    23199|             1|         0|      0|
|     16133|    22497|             4|         0|      0|
|     14918|   84926E|         

## Rollups

Un **rollup** é unha agregación multidimensional que realiza varios cálculos tipo `groupBy` nunha única operación. Permite obter subtotais xerárquicos seguindo a orde das columnas indicadas.

Se se executa:
`rollup("date", "Country")`

Spark xera agregacións para estes niveis:
- (`date`, `Country`) → detalle por data e país
- (`date`) → subtotal por data (todos os países)
- (`)`) → total xeral (todas as datas e todos os países)

No resultado aparecen valores `NULL` nas columnas de agrupación para representar eses subtotais:
- `Country = NULL` indica “subtotal por date”
- `date = NULL` indica “total xeral”
- se `date = NULL` e `Country = NULL`, trátase do total para todo o DataFrame

A continuación móstrase un exemplo calculando a suma de `Quantity`.


In [14]:
# Exemplo: Crear un rollup por date e Country e calcular a suma de Quantity

from pyspark.sql.functions import sum

# Amosamos o esquema para confirmar que existen as columnas date, Country e Quantity
print("Esquema de df_no_nulo:")
df_no_nulo.printSchema()

# Aplicamos rollup("date", "Country") e calculamos a suma de Quantity
# Renomeamos a columna agregada a total_quantity e ordenamos por date para facilitar a lectura
df_enroscado = (
    df_no_nulo.rollup("date", "Country")
        .agg(sum("Quantity"))
        .selectExpr(
            "date",
            "Country",
            "`sum(Quantity)` as total_quantity"
        )
        .orderBy("date")
)

# Imprimimos a descrición do resultado
print("Rollup por (date, Country) coa suma de Quantity (inclúe subtotais e total xeral):")

# Amosamos o resultado
df_enroscado.show()


Esquema de df_no_nulo:
root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- date: string (nullable = true)

Rollup por (date, Country) coa suma de Quantity (inclúe subtotais e total xeral):


[Stage 25:=============================>                            (1 + 1) / 2]

+----------+--------------+--------------+
|      date|       Country|total_quantity|
+----------+--------------+--------------+
|      NULL|          NULL|       4906888|
|2010-12-01|     Australia|           107|
|2010-12-01|   Netherlands|            97|
|2010-12-01|          EIRE|           243|
|2010-12-01|       Germany|           117|
|2010-12-01|United Kingdom|         21167|
|2010-12-01|          NULL|         24032|
|2010-12-01|        France|           449|
|2010-12-01|        Norway|          1852|
|2010-12-02|United Kingdom|         20705|
|2010-12-02|          EIRE|             4|
|2010-12-02|       Germany|           146|
|2010-12-02|          NULL|         20855|
|2010-12-03|      Portugal|            65|
|2010-12-03|          NULL|         11548|
|2010-12-03|        Poland|           140|
|2010-12-03|          EIRE|          2375|
|2010-12-03|       Belgium|           528|
|2010-12-03|       Germany|           170|
|2010-12-03|   Switzerland|           110|
+----------

Os valores `NULL` nas columnas de agrupación representan subtotais.

- Cando `Country` é `NULL`, a fila representa o subtotal por `date` (sumando todos os países dese día).
- Cando `date` é `NULL`, a fila representa o total xeral (todas as datas).
- Cando `date` e `Country` son `NULL`, é o total xeral para todo o DataFrame.


In [15]:
# Exemplo: Ver as filas onde Country é NULL (subtotais por date)

resultado = df_enroscado.where("Country IS NULL")

print("Filas do rollup onde Country é NULL (subtotal por date):")
resultado.show()


Filas do rollup onde Country é NULL (subtotal por date):


[Stage 28:=============================>                            (1 + 1) / 2]

+----------+-------+--------------+
|      date|Country|total_quantity|
+----------+-------+--------------+
|      NULL|   NULL|       4906888|
|2010-12-01|   NULL|         24032|
|2010-12-02|   NULL|         20855|
|2010-12-03|   NULL|         11548|
|2010-12-05|   NULL|         16394|
|2010-12-06|   NULL|         16095|
|2010-12-07|   NULL|         19351|
|2010-12-08|   NULL|         21275|
|2010-12-09|   NULL|         16904|
|2010-12-10|   NULL|         15388|
|2010-12-12|   NULL|         10561|
|2010-12-13|   NULL|         15234|
|2010-12-14|   NULL|         17108|
|2010-12-15|   NULL|         18169|
|2010-12-16|   NULL|         29482|
|2010-12-17|   NULL|         10517|
|2010-12-19|   NULL|          3735|
|2010-12-20|   NULL|         12617|
|2010-12-21|   NULL|         10888|
|2010-12-22|   NULL|          3053|
+----------+-------+--------------+
only showing top 20 rows



In [16]:
# Exemplo: Ver as filas onde date é NULL (total xeral)

resultado = df_enroscado.where("date IS NULL")

print("Filas do rollup onde date é NULL (total xeral):")
resultado.show()


Filas do rollup onde date é NULL (total xeral):


[Stage 31:=============================>                            (1 + 1) / 2]

+----+-------+--------------+
|date|Country|total_quantity|
+----+-------+--------------+
|NULL|   NULL|       4906888|
+----+-------+--------------+



## Cubo (`cube`)

Un **cubo** (*cube*) é un paso máis respecto ao `rollup`. Mentres que o `rollup` calcula subtotais seguindo unha orde xerárquica das columnas, o `cube` calcula agregacións para **todas as combinacións posibles** das dimensións indicadas.

Se se executa:
`cube("date", "Country")`

Spark xera agregacións para estes niveis:
- (`date`, `Country`) → detalle por data e país
- (`date`) → subtotal por data (todos os países)
- (`Country`) → subtotal por país (todas as datas)
- (`)`) → total xeral (todas as datas e todos os países)

No resultado aparecen valores `NULL` nas columnas de agrupación para representar os subtotais:
- `Country = NULL` indica “subtotal por date”
- `date = NULL` indica “subtotal por Country”
- `date = NULL` e `Country = NULL` indica o total xeral

Este tipo de operación é útil para análises multidimensionais, semellante a un escenario de OLAP.


In [17]:
# Exemplo: Crear un cubo por date e Country e calcular a suma de Quantity

from pyspark.sql.functions import col, sum

# Aplicamos cube("date", "Country") e calculamos a suma de Quantity
# Renomeamos a columna agregada e ordenamos por date para facilitar a lectura
df_cubo = (
    df_no_nulo.cube("date", "Country")
        .agg(sum(col("Quantity")).alias("total_quantity"))
        .select("date", "Country", "total_quantity")
        .orderBy("date")
)

# Imprimimos a descrición do resultado
print("Cube por (date, Country) coa suma de Quantity (inclúe todos os subtotais posibles):")

# Amosamos o resultado
df_cubo.show()


Cube por (date, Country) coa suma de Quantity (inclúe todos os subtotais posibles):


[Stage 34:=============================>                            (1 + 1) / 2]

+----+--------------------+--------------+
|date|             Country|total_quantity|
+----+--------------------+--------------+
|NULL|              France|        109848|
|NULL|         Unspecified|          1789|
|NULL|               Italy|          7999|
|NULL|              Poland|          3653|
|NULL|               Japan|         25218|
|NULL|               Malta|           944|
|NULL|      United Kingdom|       4008533|
|NULL|             Denmark|          8188|
|NULL|                NULL|       4906888|
|NULL|           Singapore|          5234|
|NULL|                 RSA|           352|
|NULL|             Germany|        117448|
|NULL|              Cyprus|          6317|
|NULL|             Finland|         10666|
|NULL|             Austria|          4827|
|NULL|United Arab Emirates|           982|
|NULL|           Australia|         83653|
|NULL|              Sweden|         35637|
|NULL|              Norway|         19247|
|NULL|         Switzerland|         29778|
+----+-----

As filas con valores `NULL` representan subtotais:

- `date = NULL` → subtotal por Country (todas as datas)
- `Country = NULL` → subtotal por date (todos os países)
- `date = NULL` e `Country = NULL` → total xeral

A continuación amósanse as filas nas que `date` é `NULL`, que corresponden aos subtotais por país e ao total xeral.


In [18]:
# Exemplo: Ver as filas onde date é NULL (subtotais por Country e total xeral)

resultado = df_cubo.where("date IS NULL")

print("Filas do cubo onde date é NULL (subtotal por Country e total xeral):")
resultado.show()


Filas do cubo onde date é NULL (subtotal por Country e total xeral):


[Stage 37:>                                                         (0 + 2) / 2]

+----+--------------------+--------------+
|date|             Country|total_quantity|
+----+--------------------+--------------+
|NULL|             Finland|         10666|
|NULL|              Poland|          3653|
|NULL|             Iceland|          2458|
|NULL|               Malta|           944|
|NULL|United Arab Emirates|           982|
|NULL|           Singapore|          5234|
|NULL|              Greece|          1556|
|NULL|             Germany|        117448|
|NULL|                 USA|          1034|
|NULL|              France|        109848|
|NULL|               Italy|          7999|
|NULL|               Japan|         25218|
|NULL|      United Kingdom|       4008533|
|NULL|      Czech Republic|           592|
|NULL|              Sweden|         35637|
|NULL|              Canada|          2763|
|NULL|             Denmark|          8188|
|NULL|                NULL|       4906888|
|NULL|                 RSA|           352|
|NULL|              Cyprus|          6317|
+----+-----

## Pivote (`pivot`)

Pivotar permite transformar valores dunha columna en **novas columnas**. É dicir, unha columna categórica (por exemplo `Country`) convértese en múltiples columnas, unha por cada valor posible.

No DataFrame de exemplo:
- agrúpase por `date`
- pívtase por `Country`
- e calcúlase unha agregación (por exemplo `sum`) para cada país

O resultado é un DataFrame onde:
- hai unha fila por `date`
- e unha columna por país (ou por cada país + métrica agregada)
- os valores son os resultados da agregación

Este tipo de operación é útil para:
- crear táboas tipo “matriz” (datas en filas, países en columnas)
- comparar facilmente categorías
- preparar datos para gráficos ou informes

É importante ter en conta que:
- se hai moitos países distintos, o número de columnas pode medrar moito
- por rendemento, é habitual pivotar só sobre un subconxunto de valores cando hai alta cardinalidade


In [19]:
# Exemplo: Pivotar por Country e sumar todas as columnas numéricas por cada date

# Agrupamos por date, pivotamos por Country e aplicamos sum() ás columnas numéricas
pivotado = df_con_fecha.groupBy("date").pivot("Country").sum()

# Amosamos o esquema para ver as columnas xeradas polo pivote
print("Esquema do DataFrame pivotado (unha columna por país e métrica):")
pivotado.printSchema()


Esquema do DataFrame pivotado (unha columna por país e métrica):
root
 |-- date: string (nullable = true)
 |-- Australia_sum(Quantity): long (nullable = true)
 |-- Australia_sum(UnitPrice): double (nullable = true)
 |-- Australia_sum(CustomerID): long (nullable = true)
 |-- Austria_sum(Quantity): long (nullable = true)
 |-- Austria_sum(UnitPrice): double (nullable = true)
 |-- Austria_sum(CustomerID): long (nullable = true)
 |-- Bahrain_sum(Quantity): long (nullable = true)
 |-- Bahrain_sum(UnitPrice): double (nullable = true)
 |-- Bahrain_sum(CustomerID): long (nullable = true)
 |-- Belgium_sum(Quantity): long (nullable = true)
 |-- Belgium_sum(UnitPrice): double (nullable = true)
 |-- Belgium_sum(CustomerID): long (nullable = true)
 |-- Brazil_sum(Quantity): long (nullable = true)
 |-- Brazil_sum(UnitPrice): double (nullable = true)
 |-- Brazil_sum(CustomerID): long (nullable = true)
 |-- Canada_sum(Quantity): long (nullable = true)
 |-- Canada_sum(UnitPrice): double (nullable = true

O DataFrame pivotado xera columnas para cada país e para cada medida numérica agregada.

Por exemplo, poden aparecer columnas do estilo:
- `USA_sum(Quantity)`
- `USA_sum(UnitPrice)`
- `France_sum(Quantity)`
- etc.

Como os nomes levan parénteses, en `select()` é necesario empregar comiñas invertidas (backticks) para referenciar a columna.


In [20]:
# Exemplo: Consultar unha columna concreta do pivote para un país (USA) e unha métrica (sum(Quantity))

# Filtramos por datas e seleccionamos a columna agregada de USA
resultado = (
    pivotado.where("date > '2011-12-05'")
            .select("date", "`USA_sum(Quantity)`")
)

# Imprimimos a descrición do resultado
print("Suma de Quantity para USA por date (despois do pivote):")

# Amosamos o resultado
resultado.show()


Suma de Quantity para USA por date (despois do pivote):


[Stage 48:>                                                         (0 + 2) / 2]

+----------+-----------------+
|      date|USA_sum(Quantity)|
+----------+-----------------+
|2011-12-06|             NULL|
|2011-12-09|             NULL|
|2011-12-08|             -196|
|2011-12-07|             NULL|
+----------+-----------------+



Un dos principais problemas do uso de `pivot()` é a **alta cardinalidade** da columna pivotada.  
Se unha columna ten moitos valores distintos (por exemplo, centos de países), o resultado terá centos de columnas, o que pode:

- aumentar moito o consumo de memoria
- degradar o rendemento
- xerar DataFrames difíciles de manexar

Por este motivo, é habitual **limitar explicitamente os valores do pivote** e **controlar os valores nulos** resultantes.


In [21]:
# Exemplo: Pivotar só para un subconxunto de países e controlar nulos

# Definimos explicitamente os países que nos interesan
paises = ["United Kingdom", "France", "USA"]

# Pivotamos só para eses países e calculamos a suma de Quantity
pivotado_limitado = (
    df_con_fecha.groupBy("date")
        .pivot("Country", paises)
        .sum("Quantity")
)

print("Esquema do pivote limitado a certos países:")
pivotado_limitado.printSchema()

# Enchemos os valores nulos con 0 (cando nunha data non hai rexistros dese país)
pivotado_limitado = pivotado_limitado.fillna(0)

print("Mostra do pivote limitado con nulos substituídos por 0:")
pivotado_limitado.show(5)


Esquema do pivote limitado a certos países:
root
 |-- date: string (nullable = true)
 |-- United Kingdom: long (nullable = true)
 |-- France: long (nullable = true)
 |-- USA: long (nullable = true)

Mostra do pivote limitado con nulos substituídos por 0:


[Stage 54:=============================>                            (1 + 1) / 2]

+----------+--------------+------+---+
|      date|United Kingdom|France|USA|
+----------+--------------+------+---+
|2011-01-27|          9171|    -9|  0|
|2011-01-23|          5068|   126|  0|
|2011-01-25|         11873|   -15|  0|
|2011-08-16|         11052|     0|  0|
|2011-06-26|          3023|     0|  0|
+----------+--------------+------+---+
only showing top 5 rows

